In [1]:
# Imports
uk = '+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 \
+x_0=400000 +y_0=-100000 +ellps=airy \
+towgs84=446.448,-125.157,542.06,0.15,0.247,0.842,-20.489 +units=m +no_defs'

from tqdm import tqdm as tqdmn
import pandas as pd
from geopandas import GeoDataFrame
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from geopandas import sjoin
import pickle
import re;pat = re.compile(r'''(-*\d+\.\d+ -*\d+\.\d+);*''');new_geo=[]
import warnings
import sys
warnings.filterwarnings("ignore")
sys.path.append('/warehouse/COMPLEXNET/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/')
from IPython.display import clear_output
from pyproj import transform,Proj
import numpy as np
sys.path.append('/warehouse/COMPLEXNET/jlevyabi/network_representation/python_scripts/')
import helpers_ses_prediction as hsp
from helpers_locs_to_home import time_2_date
from datetime import datetime
from collections import Counter
from scipy.spatial import cKDTree
from collections import Counter
import unidecode

Reminder: Not running filters based on users.db


##### Geolocated Users

In [16]:
#Geolocated Profiles
fgeo="/warehouse/COMPLEXNET/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/"
prof_geolocated=pickle.load(open(fgeo+"all_together_profiles.p","rb"))
data_usr_description=pd.DataFrame([(usr[0].id,usr[0].description) for usr in prof_geolocated],
                                 columns=["usr_id","desc"])
# Geolocations
dgeo=pd.read_csv(header=None,sep="\t",
                 filepath_or_buffer="/warehouse/COMPLEXNET/jlevyabi/ml_soc_econ/data_files/2014-2015_locs.txt")
dgeo.columns=["usr","time","lat","lon","text"]
fechas,days,hours,minutes,seconds,years,months=time_2_date(dgeo.time)
dgeo['day']=days;dgeo['hour']=hours;dgeo['min']=minutes;dgeo['sec']=seconds
dgeo['year']=years;dgeo['month']=months;dgeo['fecha']=fechas
france=Polygon([[-4.9658203125,42.3585439175],[8.4375,42.3585439175],
                [8.4375,51.2344073516],[-4.9658203125,51.2344073516],
                [-4.9658203125,42.3585439175]])
locs=[Point((lon,lat)) for lon,lat in zip(dgeo.lon,dgeo.lat)]
dgeo_france=dgeo[[france.contains(geo_pt) for geo_pt in locs]]

dec_prec=5
nb_rep_thresh=5
pre_fake_locs=pd.read_csv("/warehouse/COMPLEXNET/jlevyabi/geoloc/txt_files/fake_locs.txt",
                          names=["pre"])
fake_locs=[]
for x in pre_fake_locs.pre:
    try:
        lat,lon=float(x.split("-")[0]),float(x.split("-")[-1])
    except:
        continue
    if len(x)==3:
        fake_locs.append(str((lat,-lon)))
    else:
        fake_locs.append(str((lat,lon)))

fake_locs=list(set(fake_locs))
pre_fake=Counter([(round(x[0],dec_prec),round(x[1],dec_prec)) for x in  dgeo_france[["lat","lon"]].values])
set_toher_fakes=set([x for x in pre_fake if pre_fake[x]>nb_rep_thresh])
dic_final_not_nan=pickle.load(  open("/warehouse/COMPLEXNET/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/all_together_dic.p","rb"))


home_most_freq_all=go_through_home_candidates(dic_final_not_nan,take_most_frequent)
home_most_freq_night=go_through_home_candidates(dic_final_not_nan,take_most_frequent_night)

In [23]:
dic_all_users_inf_no_fake_insee={k:v for k,v in  home_most_freq_all.items()
                           if not((round(v["lat"],dec_prec),
                                   round(v["lon"],dec_prec))
                                  in set_toher_fakes) }

print("Number of different locations ... %d"%len(pre_fake))
print("Number of repeated (=fake) locations ... %d"%len(set_toher_fakes))
print("Number of geolocated users ... %d"%len(home_most_freq_all))
print("Number of users not geolocated to fake locations... %d"%len(dic_all_users_inf_no_fake_insee))

Number of different locations ... 1078482
Number of repeated (=fake) locations ... 20579
Number of geolocated users ... 22520
Number of users not geolocated to fake locations... 19839


In [32]:
geol_home_df=pd.DataFrame([(v.usr,v.lat,v.lon,v.income,v.owner_ratio)
                           for k,v in dic_all_users_inf_no_fake_insee.items()],
                         columns=["usr","lat","lon","income","or"])

##### LinkedIn Users

In [33]:
geol_home_df.head(1)

,usr,lat,lon,income,or
0,878291414,47.116211,2.360299,27799.9375,0.3125


In [42]:
import pandas as pd 
import pickle,tweepy
import re,numpy as np

# LinkedIn users
label_data=pd.read_csv(error_bad_lines=False,
                       filepath_or_buffer="/warehouse/COMPLEXNET/jlevyabi/ml_soc_econ/data_files/UKSOC_rep/linkedin/linkedin_data/full_linkedin_sofar.csv")

regex_sals = re.compile('[^0-9]')
treat_sal=lambda x: regex_sals.sub("",x) if type(x) is str else ""
salaries_per_usr=[]
for it,row in  label_data[["Salary 1","Salary 2","Salary 3","Salary 4"]].iterrows():
    sal1,sal2,sal3,sal4=map(treat_sal,row.values.tolist())
    all_sals=[]
    if type(sal1) is str and len(sal1)>0:all_sals.append(int(sal1))
    if type(sal2) is str and len(sal2)>0:all_sals.append(int(sal2))
    if type(sal3) is str and len(sal3)>0:all_sals.append(int(sal3))
    if type(sal4) is str and len(sal4)>0:all_sals.append(int(sal4))
    if len(all_sals)>0:
        salaries_per_usr.append(np.min(all_sals))
    else:
        salaries_per_usr.append(np.nan)
salaries_per_usr=np.array(salaries_per_usr)
label_data["estimated_sal"]=salaries_per_usr


In [43]:
pd.merge(label_data,geol_home_df,
         left_on="twitter_id",right_on="usr")[["twitter_id","Title",
                                               "income","estimated_sal"]].drop_duplicates("")

,twitter_id,Title,income,estimated_sal
0,399984229,Étudiant au centre polymétier,20642.421622,NaN
1,12073732,Chargé de la veille et de la prospective à la ...,17376.389610,33585.0
2,1152230846,Chargée de communication - Europe 1,21237.163876,50569.0
6,454343377,Data Analyst intern at Canal TP,17518.491429,NaN
7,465832857,Chef de publicité chez LM Work and Roll®,21981.489540,30754.0
8,473733086,Senior Director Digital Communication at Volvo...,24206.285086,30754.0
9,47969383,Data Marketing | Social Selling Practitioner |...,24561.303020,50569.0
10,127854864,Data Marketing | Social Selling Practitioner |...,24561.303020,50569.0
11,48657558,Chef de projet Marketing Digital & e-Commerce ...,18614.403198,30754.0
12,5707942,Head of Reputation & Social Media Management e...,23829.452609,30754.0


___TOO FEW USERS: USE PROFILE DESCRIPTION TO ASSOCIATE JOB____

###### APPENDIX

In [2]:
def proj_arr(points,proj_to):
    inproj = Proj(init='epsg:4326')
    outproj = Proj(proj_to)
    func = lambda x: transform(inproj,outproj,x[1],x[0])
    return np.array(list(map(func, points)))

def take_most_frequent(geopandas_usr,min_times=3,min_days=5):
    polys_visited=list(geopandas_usr.idINSPIRE)
    time_of_visit=[datetime(int(row.year),int(row.month),int(row.fecha),
                            int(row.hour),int(row.minu),int(row.sec))
                   for it,row in geopandas_usr.iterrows()]
    locat_mode=Counter(polys_visited).most_common(1)[0][0]
    nb_times=Counter(polys_visited).most_common(1)[0][1]
    inter_idx=np.where([x==locat_mode for x in polys_visited])[0].tolist()
    time_diff=[time_of_visit[i] for i in inter_idx]
    if nb_times>=min_times and (max(time_diff)-min(time_diff)).days>=min_days:
        idx_mode=polys_visited.index(locat_mode)
        return idx_mode,geopandas_usr.iloc[idx_mode][["lat","lon"]]
    else:
        return None,None

def get_check_in_rate_margin_most_freq(geopandas_usr):
    polys_visited=list(geopandas_usr.idINSPIRE)
    inter=Counter(polys_visited).most_common(2)
    if len(inter)<2:
        return None,None,None,None,None
    locat_mode,sec_locat_mode=inter
    idx_mode,idx_mode_sec=polys_visited.index(locat_mode[0]),polys_visited.index(sec_locat_mode[0])
    return (idx_mode,geopandas_usr.iloc[idx_mode][["lat","lon"]],
            idx_mode_sec,geopandas_usr.iloc[idx_mode_sec][["lat","lon"]],
           ((locat_mode[1]+0.0-sec_locat_mode[1])/(sec_locat_mode[1]+locat_mode[1])))

def take_most_frequent_night(geopandas_usr,start=21,stop=6) :
    polys_visited=(geopandas_usr.idINSPIRE)
    polys_visited_night=polys_visited[(geopandas_usr.hour>=start)|(geopandas_usr.hour<stop)]
    if len(polys_visited_night)==0:
        return None,None
    locat_mode=Counter(polys_visited_night).most_common(1)[0][0]
    idx_mode=list(polys_visited).index(locat_mode)
    return idx_mode,geopandas_usr.iloc[idx_mode][["lat","lon"]]

def get_distance_matrix(geopandas_usr):
    x = np.array(geopandas_usr[["lat","lon"]]).astype(float).tolist()
    y=proj_arr(x,uk)
    ztree = cKDTree(y)
    z = ztree.sparse_distance_matrix(ztree,1e6,p=2).todense()
    return z

def distance_to_home(geopandas_usr,select_home_loc,args):
    idx,loc=select_home_loc(geopandas_usr,*args)
    if idx is None:
        return None,None,None
    mat_dist=get_distance_matrix(geopandas_usr)
    return mat_dist[idx,:].tolist()[0],list(geopandas_usr.day),list(geopandas_usr.hour)

def go_through_home_candidates(dic_gpd,select_home_loc):
    dic_exam={}
    for usr,gpd in tqdmn(dic_gpd.items()):
        idx,loc=select_home_loc(gpd)
        if idx is None:
            continue
        dic_exam.setdefault(usr,gpd.iloc[idx])
    return dic_exam

def go_through_geol_users(dic_gpd,select_home_loc,args,outlier_lim=6e4):
    dic_per_day={k:np.zeros(24) for k in range(7)}
    dic_nb_per_day={k:np.zeros(24) for k in range(7)}
    dic_exam={}
    loss=[]
    for usr,gpd in tqdmn(dic_gpd.items()):
        dic_exam.setdefault(usr,[])
        dists,days,hours=distance_to_home(gpd,select_home_loc,args)
        new_dists=np.array(dists)
        if dists is None:
            continue
        loss.append(1-(np.sum([new_dists<outlier_lim])+0.0)/len(dists) )
        dists=new_dists[new_dists<outlier_lim]
        for dist,day,hour in zip(dists,days,hours):
            dic_exam[usr].append(dist)
            dic_per_day[day][hour]+=dist
            dic_nb_per_day[day][hour]+=1
    dic_day={}
    for k,v in dic_per_day.items():
        dic_day[k]=(v/dic_nb_per_day[k])/100
    return dic_day,dic_exam,loss